### AI Agent Tool Call Testing

In [4]:
from deepeval.tracing import observe

#### AI Agent with Tools

In [5]:
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

s_tool = DuckDuckGoSearchRun()

@tool
@observe(type="tool")
def search_tool(query: str):
    "Tool to search online for given query and return result"
    return s_tool.run(query)

@tool
@observe(type="tool")
def add_numbers(a: int, b: int):
    "Add two numbers and returns result"
    return int(a) + int(b)

@tool
@observe(type="tool")
def subtract_numbers(a: int, b: int):
    "Subtract two numbers and returns result"
    return int(a) - int(b)

tools = [add_numbers, subtract_numbers, search_tool]

#### Local LLMs

In [6]:
from langchain_ollama import ChatOllama
@observe(type="llm",
         name="Local LLM",
         model="qwen2.5:latest")
def local_llm():
    llm = ChatOllama(base_url="http://localhost:11434", 
                      model="qwen2.5:latest", 
                      temperature=0.5, 
                      max_token=250)
    return llm.bind_tools(tools=tools)
llm = local_llm()

#### Create ChatPrompt, agent

In [ ]:
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant with access to tools."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)
result = agent_executor.invoke({"input": "What is 10 + 5? and who is the president of USA", "chat_history": []})
print(result)




> Entering new AgentExecutor chain...

Invoking: `add_numbers` with `{'a': 10, 'b': 5}`


15
Invoking: `search_tool` with `{'query': 'president of USA'}`


Upon the resignation of 37th president , Richard Nixon, Gerald Ford became the 38th president even though he simply served out the remainder of ... Unrelated to and quite dissimilar from the later office of president of the United States, it was a largely ceremonial position with no executive ... President Trump built on his success in private life when he entered into politics and public service. He remarkably won the Presidency in his first ever run for any political office. Jan 24, 2025 · President Trump announced key appointments to the White House Office of Communications, Public Liaison, and Cabinet Affairs, which will be overseen by Deputy Chief … Nov 6, 2025 · On May 12, 2025, President Trump signed an Executive Order titled: “Delivering Most-Favored-Nation Prescription Drug Pricing to American Patients” directing the …

[Confident AI Trace Log]  Successfully posted trace (0 traces remaining in queue, 2 in flight): 
https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/observatory/traces/90d99c47-6964-4df9-b15c-16933fa08
946 
To disable dev logging, set CONFIDENT_TRACE_VERBOSE=0 as an environment variable.

[Confident AI Trace Log]  Successfully posted trace (0 traces remaining in queue, 1 in flight): 
https://app.confident-ai.com/project/cmiou0pbn01p0qh1fbxhly3fa/observatory/traces/b04c24b7-4809-4eb7-989e-34106b23c
d6e 
To disable dev logging, set CONFIDENT_TRACE_VERBOSE=0 as an environment variable.

The sum of 10 and 5 is 15.

As of my last update, the president of the United States was Donald Trump. However, please note that this information might be outdated as it refers to a past event. For the most current information, I would recommend checking a reliable news source or the official White House website.

> Finished chain.
{'input': 'What is 10 + 5? and who is the president of USA', 'chat_history': [], 'output': 'The sum of 10 and 5 is 15.\n\nAs of my last update, the president of the United States was Donald Trump. However, please note that this information might be outdated as it refers to a past event. For the most current information, I would recommend checking a reliable news source or the official White House website.'}
